# Notebook 02: Pipeline for ANODE results

This notebook goes through the pipeline for obtaining results using the ANODE (Anomaly Detection with Density Estimation) method.

In [1]:
import os
import argparse
import numpy as np

from run_ANODE_training import main as train_DE
from run_classifier_data_creation import main as create_data
from run_classifier_training import main as train_classifier
from run_ANODE_evaluation import main as eval_ANODE
from evaluation_utils import full_single_evaluation, classic_ANODE_eval, minimum_val_loss_model_evaluation

In [2]:
mode = 'ANODE'
data_dir = '../separated_data'
save_dir = 'ANODE_models'
# Shift on jet mass variables to be applied.
datashift = 0.
# Shift is not correlated to the actual mjj but randomized.
random_shift = False
# Whether to apply an (ANODE paper) fiducial cut on the data (and samples).
fiducial_cut = False
# Suppress the processing of the extra signal sample.
no_extra_signal = True
verbose = False

# ANODE model config file (.yml).
DE_config_file = '../DE_MAF_model.yml'
# 'Number of Density Estimation training epochs.'
DE_epochs = 100
# Batch size during density estimation training.
DE_batch_size = 256
# Skips the density estimation (loads existing files instead).
DF_skip = False
# Turns off the logit transform in the density estimator.
DE_no_logit = False

# File name for the density estimator.
DE_file_name = 'my_ANODE_model'

# Classifier model config file (.yml).
cf_config_file = '../classifier.yml'

# Number of classifier training epochs
cf_epochs = 100
# Number of samples to be generated. Currently the samples will be cut down to match data proportion.
cf_n_samples = 130000

# Sample the conditional from a KDE fit rather than a uniform distribution.
cf_realistic_conditional = False
# Bandwith of the KDE fit (used when realistic_conditional is selected)
cf_KDE_bandwidth = 0.01
# Add the full number of samples to the training set rather than mixing it in equal parts with data.
cf_oversampling = False
# Turns off logit tranform in the classifier.
cf_no_logit = False
# Space-separated list of pre-sampled npy files of physical variables if the sampling has been done externally. The format is 
# (mjj, mj1, dmj, tau21_1, tau21_2)
cf_external_samples = ""
# Lower boundary of signal region.
cf_SR_min = 3.3
# Upper boundary of signal region.
cf_SR_max = 3.7
# Number of independent classifier training runs.
cf_n_runs = 10
# Batch size during classifier training.
cf_batch_size = 128
# Use the conditional variable as classifier input during training.
cf_use_mjj = False
# Weight the classes according to their occurence in the training set. 
# Necessary if the training set was intentionally oversampled.'
cf_use_class_weights = False
# Central value of signal region. Must only be given for using CWoLa with weights.
cf_SR_center = 3.5
# Make use of extra background (for supervised and idealized AD).
cf_extra_bkg = False
# Define a separate validation set to pick the classifier epochs.
cf_separate_val_set = False
# Save the tensorflow model after each epoch instead of saving predictions.
cf_save_model = True
# Skips the creation of the classifier dataset (loads existing files instead).
cf_skip_create = False
# Skips the training of the classifier (loads existing files instead).
cf_skip_train = False

In [3]:
import argparse
import os
import torch
import numpy as np
import pickle
from data_handler import LHCORD_data_handler
from ANODE_training_utils import train_ANODE, plot_ANODE_losses
from density_estimator import DensityEstimator

In [4]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [5]:
def prepare_processing_dict(source):
    out = {}
    out['max'] = source['max']
    out['min'] = source['min']
    out['mean2'] = source['mean2']
    out['std2'] = source['std2']
    out['std2_logit_fix'] = source['std2_logit_fix']
    return out

In [23]:
def train_DE(**kwargs):
    
    # for debugging:
    # torch.manual_seed(2104)
    # np.random.seed(2104)

    # selecting appropriate device
    CUDA = torch.cuda.is_available()
    print("cuda available:", CUDA)
    device = torch.device("cuda:0" if CUDA else "cpu")

    # checking for data separation
    data_files = os.listdir(kwargs['data_dir'])
    if "innerdata_val.npy" in data_files:
        finer_data_split = True
    else:
        finer_data_split = False

    if finer_data_split:
        innerdata_val_path = os.path.join(kwargs['data_dir'], 'innerdata_val.npy')
    else:
        innerdata_val_path = os.path.join(kwargs['data_dir'], 'innerdata_test.npy')

    # data preprocessing
    data = LHCORD_data_handler(os.path.join(kwargs['data_dir'], 'innerdata_train.npy'),
                               innerdata_val_path,
                               os.path.join(kwargs['data_dir'], 'outerdata_train.npy'),
                               os.path.join(kwargs['data_dir'], 'outerdata_test.npy'),
                               None,
                               batch_size=kwargs['batch_size'],
                               device=device)
    if kwargs['datashift'] != 0:
        print("applying a datashift of", kwargs['datashift'])
        data.shift_data(kwargs['datashift'], constant_shift=False, random_shift=kwargs['random_shift'],
                        shift_mj1=True, shift_dm=True, additional_shift=False)
    data.preprocess_ANODE_data(no_logit=kwargs['no_logit'],
                               no_mean_shift=kwargs['no_logit'])
    if kwargs['inner_model']:
        train_loader = data.inner_ANODE_datadict_train['loader']
        test_loader = data.inner_ANODE_datadict_test['loader']
        data_std = data.inner_ANODE_datadict_train['std2_logit_fix']
        train_data_processing = prepare_processing_dict(data.inner_ANODE_datadict_train)
    else:
        train_loader = data.outer_ANODE_datadict_train['loader']
        test_loader = data.outer_ANODE_datadict_test['loader']
        data_std = data.outer_ANODE_datadict_train['std2_logit_fix']
        train_data_processing = prepare_processing_dict(data.outer_ANODE_datadict_train)
    pickle.dump(train_data_processing, open(os.path.join(kwargs['savedir'], 'data_processing.p'), 'wb'))

    # actual training
    anode = DensityEstimator(kwargs['config_file'], device=device,
                             verbose=kwargs['verbose'], bound=kwargs['no_logit'])
    model, optimizer = anode.model, anode.optimizer

    train_ANODE(model, optimizer, train_loader, test_loader, kwargs['model_file_name'],
                kwargs['epochs'], savedir=kwargs['savedir'], device=device, verbose=kwargs['verbose'],
                no_logit=kwargs['no_logit'], data_std=data_std)

    # plot losses
    train_losses = np.load(os.path.join(kwargs['savedir'], kwargs['model_file_name']+"_train_losses.npy"))
    val_losses = np.load(os.path.join(kwargs['savedir'], kwargs['model_file_name']+"_val_losses.npy"))
    plot_ANODE_losses(train_losses, val_losses, yrange=None,
                      savefig=os.path.join(kwargs['savedir'], kwargs['model_file_name']+"_loss_plot"),
                      suppress_show=True)

In [6]:
def find_best_epochs(num_models, **kwargs):
    """ looks through saved val-losses and creates list-of-paths of num_models best ones"""
    val_losses = np.load(os.path.join(kwargs['savedir'], kwargs['model_file_name']+"_val_losses.npy"))
    idx = np.argpartition(val_losses, num_models)[:num_models] #faster than argsort
    ret_list = []
    for index in idx:
        ret_list.append(os.path.join(kwargs['savedir'],
                                     kwargs['model_file_name']+'_epoch_'+str(index-1)+'.par'))
    return ret_list

In [7]:
outer_DE_kwargs = {
    'config_file': DE_config_file,
    'data_dir': data_dir,
    'savedir': save_dir,
    'datashift': datashift,
    'random_shift': random_shift,
    'verbose': verbose,
    'model_file_name': DE_file_name,
    'epochs': DE_epochs,
    'batch_size': DE_batch_size,
    'no_logit': DE_no_logit,
    'inner_model': False
}

In [24]:
train_DE(**outer_DE_kwargs)

cuda available: False
DensityEstimator has 274800 parameters
n_nans = 0
n_highs = 0
n_nans = 0
n_highs = 0
train_loss =  6.018267997520982
val_loss =  6.013983478417268

Epoch: 0
n_nans = 0
n_highs = 0
train_loss =  5.349560940641482
val_loss =  5.340720637424572

Epoch: 1
n_nans = 0
n_highs = 0
train_loss =  5.32577693343789
val_loss =  5.336835973971599

Epoch: 2
n_nans = 0
n_highs = 0
train_loss =  5.322791612410083
val_loss =  5.334284134813257

Epoch: 3
n_nans = 0
n_highs = 0
train_loss =  5.320918846158794
val_loss =  5.3326866884489315

Epoch: 4
n_nans = 0
n_highs = 0
train_loss =  5.31986520000115
val_loss =  5.332341413240175

Epoch: 5
n_nans = 0
n_highs = 0
train_loss =  5.319095360489427
val_loss =  5.331729437853839

Epoch: 6
n_nans = 0
n_highs = 0
train_loss =  5.318642751069199
val_loss =  5.332231060878651

Epoch: 7
n_nans = 0
n_highs = 0
train_loss =  5.3180358588974395
val_loss =  5.331534891515164

Epoch: 8
n_nans = 0
n_highs = 0
train_loss =  5.317151847994957
val_lo

n_nans = 0
n_highs = 0
train_loss =  5.2969637451593234
val_loss =  5.329407621074367

Epoch: 85
n_nans = 0
n_highs = 0
train_loss =  5.296397415314254
val_loss =  5.329329874064471

Epoch: 86
n_nans = 0
n_highs = 0
train_loss =  5.296166113916534
val_loss =  5.327750911583772

Epoch: 87
n_nans = 0
n_highs = 0
train_loss =  5.296730256406212
val_loss =  5.329054233190176

Epoch: 88
n_nans = 0
n_highs = 0
train_loss =  5.296054429827397
val_loss =  5.32963599385442

Epoch: 89
n_nans = 0
n_highs = 0
train_loss =  5.296132431594988
val_loss =  5.328730083800651

Epoch: 90
n_nans = 0
n_highs = 0
train_loss =  5.2954361645852766
val_loss =  5.330135455002656

Epoch: 91
n_nans = 0
n_highs = 0
train_loss =  5.295737969847842
val_loss =  5.328779874621211

Epoch: 92
n_nans = 0
n_highs = 0
train_loss =  5.295417457523124
val_loss =  5.32986006543443

Epoch: 93
n_nans = 0
n_highs = 0
train_loss =  5.295435175198749
val_loss =  5.328405583227003

Epoch: 94
n_nans = 0
n_highs = 0
train_loss =  5.2

In [8]:
inner_DE_kwargs = {
    'config_file': DE_config_file,
    'data_dir': data_dir,
    'savedir': save_dir,
    'datashift': datashift,
    'random_shift': random_shift,
    'verbose': verbose,
    'model_file_name': f"{DE_file_name}_inner",
    'epochs': DE_epochs,
    'batch_size': DE_batch_size,
    'no_logit': DE_no_logit,
    'inner_model': True
}

In [28]:
train_DE(**inner_DE_kwargs)

cuda available: False
DensityEstimator has 274800 parameters
n_nans = 0
n_highs = 0
n_nans = 0
n_highs = 0
train_loss =  6.72812008147544
val_loss =  6.735505839188893

Epoch: 0
n_nans = 0
n_highs = 0
train_loss =  5.431384976137612
val_loss =  5.414118905862172

Epoch: 1
n_nans = 0
n_highs = 0
train_loss =  5.36897210747245
val_loss =  5.392220775286357

Epoch: 2
n_nans = 0
n_highs = 0
train_loss =  5.355431184626228
val_loss =  5.384280363718669

Epoch: 3
n_nans = 0
n_highs = 0
train_loss =  5.349065154724304
val_loss =  5.381832480430603

Epoch: 4
n_nans = 0
n_highs = 0
train_loss =  5.346202893926936
val_loss =  5.380090534687042

Epoch: 5
n_nans = 0
n_highs = 0
train_loss =  5.343728252132573
val_loss =  5.37657876809438

Epoch: 6
n_nans = 0
n_highs = 0
train_loss =  5.340754400828082
val_loss =  5.374277710914612

Epoch: 7
n_nans = 0
n_highs = 0
train_loss =  5.339749721720359
val_loss =  5.375045398871104

Epoch: 8
n_nans = 0
n_highs = 0
train_loss =  5.3390409229548395
val_loss

n_nans = 0
n_highs = 0
train_loss =  5.327229837524216
val_loss =  5.371967176596324

Epoch: 85
n_nans = 0
n_highs = 0
train_loss =  5.326993250578182
val_loss =  5.371104180812836

Epoch: 86
n_nans = 0
n_highs = 0
train_loss =  5.327501661580437
val_loss =  5.3723752697308855

Epoch: 87
n_nans = 0
n_highs = 0
train_loss =  5.327209347067775
val_loss =  5.37042897939682

Epoch: 88
n_nans = 0
n_highs = 0
train_loss =  5.326211226323127
val_loss =  5.373775084813436

Epoch: 89
n_nans = 0
n_highs = 0
train_loss =  5.326764439493572
val_loss =  5.3740357756614685

Epoch: 90
n_nans = 0
n_highs = 0
train_loss =  5.325860715562654
val_loss =  5.3715522686640425

Epoch: 91
n_nans = 0
n_highs = 0
train_loss =  5.327171716149363
val_loss =  5.37151962518692

Epoch: 92
n_nans = 0
n_highs = 0
train_loss =  5.326503750454631
val_loss =  5.371562937895457

Epoch: 93
n_nans = 0
n_highs = 0
train_loss =  5.326939739800364
val_loss =  5.370597004890442

Epoch: 94
n_nans = 0
n_highs = 0
train_loss =  5.

In [9]:
outer_best = find_best_epochs(10, **outer_DE_kwargs)
inner_best = find_best_epochs(10, **inner_DE_kwargs)

In [12]:
eval_kwargs = {
    'savedir': save_dir,
    'datashift': datashift,
    'data_dir': data_dir,
    'random_shift': random_shift,
    'config_file': DE_config_file,
    'fiducial_cut': fiducial_cut,
    'no_extra_signal': no_extra_signal,
    'inner_models': inner_best,
    'outer_models': outer_best
}

In [16]:
import os
import argparse
import torch
from data_handler import LHCORD_data_handler
from density_estimator import DensityEstimator
from ANODE_evaluation_utils import compute_ANODE_score

@torch.no_grad()
def eval_ANODE(**kwargs):

    # selecting appropriate device
    CUDA = torch.cuda.is_available()
    print("cuda available:", CUDA)
    device = torch.device("cuda:0" if CUDA else "cpu")

    # checking for data separation
    data_files = os.listdir(kwargs['data_dir'])
    if not kwargs['no_extra_signal']:
        if "innerdata_extrasig_test.npy" in data_files:
            extrasig_path = os.path.join(kwargs['data_dir'], 'innerdata_extrasig_test.npy')
        elif "innerdata_extrasig.npy" in data_files:
            extrasig_path = os.path.join(kwargs['data_dir'], 'innerdata_extrasig.npy')
    else:
        extrasig_path = None

    innerdata_test_path = [os.path.join(kwargs['data_dir'], 'innerdata_test.npy')]
    if "innerdata_extrabkg_test.npy" in data_files:
        innerdata_test_path.append(os.path.join(kwargs['data_dir'], 'innerdata_extrabkg_test.npy'))

    # data preprocessing
    data = LHCORD_data_handler(os.path.join(kwargs['data_dir'], 'innerdata_train.npy'),
                               innerdata_test_path,
                               os.path.join(kwargs['data_dir'], 'outerdata_train.npy'),
                               os.path.join(kwargs['data_dir'], 'outerdata_test.npy'),
                               extrasig_path,
                               batch_size=256,
                               device=device)
    if kwargs['datashift'] != 0:
        print("applying a datashift of", args.datashift)
        data.shift_data(kwargs['datashift'], constant_shift=False, random_shift=kwargs['random_shift'],
                        shift_mj1=True, shift_dm=True, additional_shift=False)

    data.preprocess_ANODE_data(fiducial_cut=kwargs['fiducial_cut'])
    data.preprocess_classic_ANODE_data(fiducial_cut=kwargs['fiducial_cut'])

    init_inner_model_list = []
    for model_path in kwargs['inner_models']:
        init_inner_model_list.append(DensityEstimator(kwargs['config_file'], eval_mode=True,
                                                      load_path=model_path, device=device).model)

    init_outer_model_list = []
    for model_path in kwargs['outer_models']:
        init_outer_model_list.append(DensityEstimator(kwargs['config_file'], eval_mode=True,
                                                      load_path=model_path, device=device).model)

    preds, sig_labels, preds_extrasig = compute_ANODE_score(init_inner_model_list,
                                                            init_outer_model_list, data, device,
                                                            savedir=kwargs['savedir'],
                                                            extra_signal=not kwargs['no_extra_signal'])

In [17]:
eval_ANODE(**eval_kwargs)

cuda available: False
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_76.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_38.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_44.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_33.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_68.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_80.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_58.par
DensityEstimator has 274800 parameters
Loading model parameters from ANODE_models/my_ANODE_model_inner_epoch_69.par
DensityEstimator has 274800 parameters
Loading mod

In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import torch
import sklearn
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from scipy.interpolate import interp1d

from evaluation_utils import (load_predictions, tprs_fprs_sics,
                              minumum_validation_loss_ensemble, compare_on_various_runs)
def classic_ANODE_eval(predsdir, savefig=None, suppress_show=False, extra_signal=True, return_all=False):
    preds = np.load(os.path.join(predsdir, "preds_matris.npy"))
    labels = np.load(os.path.join(predsdir, "sig_labels.npy"))
    if extra_signal:
        preds_extrasig = np.load(os.path.join(predsdir, "preds_matris_extrasig.npy"))

    # removing nans and infs
    nan_mask = np.logical_and(~np.isnan(preds), ~np.isinf(preds))
    preds = preds[:,:,nan_mask[0,0,:]]
    labels = labels[nan_mask[0,0,:]]
    if extra_signal:
        nan_mask = np.logical_and(~np.isnan(preds_extrasig), ~np.isinf(preds_extrasig))
        preds_extrasig = preds_extrasig[:,:,nan_mask[0,0,:]]

    if extra_signal:
        preds_combined = np.concatenate((preds, preds_extrasig), axis=-1)
        labels_combined = np.concatenate((labels, np.ones(preds_extrasig.shape[-1])))
    else:
        preds_combined = preds
        labels_combined = labels

    tprs, fprs, sics = tprs_fprs_sics(preds_combined, np.ones(preds_combined.shape[-1]), labels_combined.reshape(-1,1))
    fprs_list = [fprs]
    tprs_list = [tprs]
    pick_epochs_list = [np.zeros(1)]
    labels_list = [""]

    return compare_on_various_runs(tprs_list, fprs_list, pick_epochs_list, labels_list, savefig=savefig, suppress_show=suppress_show, return_all=return_all)

In [21]:
_ = classic_ANODE_eval(save_dir, extra_signal=not no_extra_signal,
                       savefig=os.path.join(save_dir, 'result_SIC'))